<a href="https://colab.research.google.com/github/ptran1203/gan_project/blob/master/notebooks/BAGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create the link from this drive folder to your drive.

---


https://drive.google.com/drive/folders/1RNJXceXkNatuAbNn-CKB8MrgaEHG5RpM?usp=sharing

In [1]:
cd /content

/content


In [2]:
%tensorflow_version 1.x
from google.colab import drive, output
drive.mount('/content/drive')

!rm -rf '/content/gan_project'
!git clone https://github.com/ptran1203/gan_project
!pip install git+https://www.github.com/keras-team/keras-contrib.git
output.clear()

In [3]:
cd gan_project

/content/gan_project


In [4]:
BASE_DIR = '/content/drive/My Drive/bagan'
DS_DIR = '/content/drive/My Drive/bagan/dataset/chest_xray'
DS_SAVE_DIR = '/content/drive/My Drive/bagan/dataset/save'
gratio_mode = 'uniform'
dratio_mode = 'uniform'

from bagan import *
from batch_gen import *
from utils import *
from tensorflow.keras.layers import LayerNormalization

def create_dir_if_any(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

class Bagan(BalancingGAN):
    def _build_common_encoder(self, image, min_latent_res=8):
        resolution = self.resolution
        channels = self.channels

        # build a relatively standard conv net, with LeakyReLUs as suggested in ACGAN
        cnn = Sequential()

        cnn.add(Conv2D(128, (5, 5), padding='same', strides=(2, 2)))
        cnn.add(LeakyReLU(alpha=0.2))
        cnn.add(Dropout(0.3))
            
        cnn.add(Conv2D(256, (5, 5), padding='same', strides=(2, 2)))
        cnn.add(LeakyReLU(alpha=0.2))
        cnn.add(Dropout(0.3))

        cnn.add(Conv2D(512, (5, 5), padding='same', strides=(2, 2)))
        cnn.add(LeakyReLU(alpha=0.2))
        cnn.add(Dropout(0.3))

        cnn.add(Flatten())

        features = cnn(image)
        return features

    # def backup_point(self, epoch): return


class BatchGen(BatchGenerator):
    to_train_classes = INVERT_CATEGORIES_MAP[:2]
    to_test_classes = list(range(81, 86))


is_test = 0
## Test batch GEN
if is_test:
    bg = BatchGen(BatchGen.TRAIN, 64, 'multi_chest', 64)
    labels = np.array([0, 0, 1, 1, 2, 2, 3 ,1])
    samples = bg.ramdom_kshot_images(4, labels)
    print(samples.shape)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
gan_epochs  = 400000
adam_lr = 0.00002
batch_size = 128
# dataset_name should be "flowers", "chest", "multi_chest"
# "chest" is binary classification, "multi_chest" using chest-xray14 dataset
dataset_name = 'multi_chest'
latent_size = 128
# Use resnet architecture for Generator
resnet = False
# Use self-attention mechanism
attention = False
np.random.seed(0)
# Image resoulution
rst = 64
is_prune = False

prune = [900, 900] if is_prune else None

res_dir = BASE_DIR + '/result/bagan_{}_{}_v2'.format(dataset_name,rst)
create_dir_if_any(res_dir)

bg_train_full = BatchGen(BatchGen.TRAIN, batch_size, dataset_name, rst,prune_classes=prune)
bg_test = BatchGen(BatchGen.TEST, batch_size, dataset_name, rst)
channels = bg_train_full.dataset_x[0].shape[-1]
shape = bg_train_full.dataset_x[0].shape

print('img shape', shape)
classes = bg_train_full.get_label_table()
target_classes = np.array(range(len(classes)))
target_class_id = 0 # train to balance this class
print('Class counters: ', bg_train_full.per_class_count)
gan = Bagan(
    target_classes,
    target_class_id,
    adam_lr=adam_lr,
    latent_size=latent_size,
    res_dir=res_dir,
    image_shape=shape,
    min_latent_res=4,
    autoenc_epochs=50
)
gan.train(bg_train_full, bg_test, epochs=gan_epochs)


In [ ]:
def build_generator(latent_size, init_resolution=8):
        resolution = 64
        channels = 1
        init_channels = 256
        cnn = Sequential()

        cnn.add(Dense(init_channels * init_resolution * init_resolution, input_dim=latent_size))
        cnn.add(BatchNormalization())
        cnn.add(LeakyReLU())
        cnn.add(Reshape((init_resolution, init_resolution, init_channels)))
       
        crt_res = init_resolution
        # upsample
        while crt_res < resolution/2:
            cnn.add(Conv2DTranspose(
                init_channels, kernel_size = 5, strides = 2, padding='same'))
            cnn.add(LeakyReLU(alpha=0.02))
            init_channels //= 2
            crt_res = crt_res * 2
            assert crt_res <= resolution,\
                "Error: final resolution [{}] must equal i*2^n. Initial resolution i is [{}]. n must be a natural number.".format(resolution, init_resolution)
        cnn.add(Conv2DTranspose(
                    1, kernel_size = 5,
                    strides = 2, padding='same',
                    activation='tanh'))

        latent = Input(shape=(latent_size, ))

        fake_image_from_latent = cnn(latent)
        return Model(inputs=latent, outputs=fake_image_from_latent)

model = build_generator(128, 4)
model.layers[1].summary()